In [1]:
import os 
import logging 
from kiteconnect import KiteConnect
import datetime as dt
import pandas as pd 
import matplotlib.pyplot as plt
import numpy as np 

In [2]:
cwd = os.chdir(r'C:\Users\rsurs\OneDrive\Documents\Zerodha\api_keys')
#print(f"current dir: {os.getcwd()}")
request_token = open('request_token.txt','r').read()
key_secret = open('credentials.txt','r').read().split()
access_token = open('access_token.txt','r').read()
api_key = key_secret[0]
api_secret = key_secret[1]
kite = KiteConnect(api_key=api_key)
kite.set_access_token(access_token=access_token)

In [3]:
instrument_dump = kite.instruments('NSE')
instrument_df = pd.DataFrame(instrument_dump)
instrument_df.to_csv('NSE_instruments.csv',index=False)

In [4]:
instrument_df.head()

,instrument_token,exchange_token,tradingsymbol,name,last_price,expiry,strike,tick_size,lot_size,instrument_type,segment,exchange
0,256265,1001,NIFTY 50,NIFTY 50,0.0,,0.0,0.0,0,EQ,INDICES,NSE
1,256777,1003,NIFTY MIDCAP 100,NIFTY MIDCAP 100,0.0,,0.0,0.0,0,EQ,INDICES,NSE
2,260105,1016,NIFTY BANK,NIFTY BANK,0.0,,0.0,0.0,0,EQ,INDICES,NSE
3,260617,1018,NIFTY 100,NIFTY 100,0.0,,0.0,0.0,0,EQ,INDICES,NSE
4,257033,1004,NIFTY DIV OPPS 50,NIFTY DIV OPPS 50,0.0,,0.0,0.0,0,EQ,INDICES,NSE


In [5]:
def instrumentLookup(instrument_df,symbol):
    """Looks up instrument token for a given script from instrument dump"""
    try:
        return instrument_df[instrument_df.tradingsymbol==symbol].instrument_token.values[0]
    except:
        return -1

In [6]:
print(instrumentLookup(instrument_df,'RELIANCE'))

738561


In [7]:
def fetchOHLC(ticker,interval,duration):
    """extracts historical data and outputs in the form of dataframe"""
    instrument = instrumentLookup(instrument_df,ticker)
    data = pd.DataFrame(kite.historical_data(instrument,dt.date.today()-dt.timedelta(duration), dt.date.today(),interval))
    data.set_index("date",inplace=True)
    return data


In [8]:
ohlc_df = fetchOHLC('ICICIBANK','5minute',5)

In [9]:
ohlc_df.head()

,open,high,low,close,volume
date,,,,,
2024-05-30 09:15:00+05:30,1097.00,1110.60,1096.90,1108.35,860424
2024-05-30 09:20:00+05:30,1108.55,1112.50,1107.95,1111.50,534024
2024-05-30 09:25:00+05:30,1111.70,1112.85,1109.15,1112.35,351948
2024-05-30 09:30:00+05:30,1112.05,1115.50,1112.00,1114.25,476156
2024-05-30 09:35:00+05:30,1114.20,1114.80,1110.55,1110.90,169134


In [10]:
ohlc_df.to_csv('ohlc.csv')

In [11]:
df = pd.read_csv('ohlc.csv')
df.set_index('date',inplace=True)
df.head()

,open,high,low,close,volume
date,,,,,
2024-05-30 09:15:00+05:30,1097.00,1110.60,1096.90,1108.35,860424
2024-05-30 09:20:00+05:30,1108.55,1112.50,1107.95,1111.50,534024
2024-05-30 09:25:00+05:30,1111.70,1112.85,1109.15,1112.35,351948
2024-05-30 09:30:00+05:30,1112.05,1115.50,1112.00,1114.25,476156
2024-05-30 09:35:00+05:30,1114.20,1114.80,1110.55,1110.90,169134


In [12]:
# def compute_ema(df,n):
#     df['sma'] = df['close'].rolling(n).mean()
#     df.dropna(inplace=True)
#     multiplier = (2 / (n + 1))
#     for i in range(len(df)):
#         if i == 1:
#             df.loc[df.index[i],'ema'] = df.loc[df.index[i - 1],'sma']
#             print(df.loc[df.index[i],'ema'])
#         if i >= 2:
#             df.loc[df.index[i],'ema'] = (df.loc[df.index[i],'close'] - df.loc[df.index[i - 1],'ema']) * multiplier + df.loc[df.index[i - 1],'ema'] 
#     return df
        

In [13]:
# compute_ema(df,n=50) 

In [14]:
# ema_df = df.copy()
# ema_df
# ema_df.to_csv('ema.csv')

In [15]:
# n = 12
# def compute_sma_manual(df, n, column_name):
#     df[column_name] = 0  # Initialize the column with zeros
    
#     for i in range(len(df)):
#         if i >= n - 1:
#             df.loc[df.index[i], column_name] = df['close'][i-n+1:i+1].sum() / n
            
#     return df


In [16]:
# def compute_ema(df,n,column_name):
#     df['sma'] = df['close'].rolling(n).mean()
#     df.dropna(inplace=True)
#     multiplier = (2 / (n + 1))
#     for i in range(1,len(df)):
#         if i == 1:
#             df.loc[df.index[i],column_name] = df.loc[df.index[i - 1],'sma']
#             print(df.loc[df.index[i],column_name])
#         else:
#             df.loc[df.index[i],column_name] = (df.loc[df.index[i],'close'] - df.loc[df.index[i - 1],column_name]) * multiplier + df.loc[df.index[i - 1],column_name] 
#     return df

# df = compute_ema(df,12,'fast_ma')
# df = compute_ema(df,26,'slow_ma')
# df['macd'] = df['fast_ma'] - df['slow_ma']
# df

In [17]:
# ema_df = compute_ema(df.copy(),n=50,column_name='ema')
# ema_df 


In [18]:
# ema_df.to_csv('ema.csv')

In [19]:
# df['fast_ma'] = compute_ema(df,n=12,column_name='fast_ma')
# df['slow_ma'] = compute_ema(df,n=26,column_name='slow_ma')
# df['macd'] = df['fast_ma'] - df['slow_ma']


In [20]:
def SMA(ser, n):
    return ser.rolling(window=n).mean()

def EMA(ser, n=9):
    multiplier = 2 / (n + 1)
    sma = SMA(ser, n)
    ema = np.full(len(ser), np.nan)
    ema[n - 1] = sma.dropna().iloc[0]
    for i in range(n, len(ser)):
        ema[i] = ((ser.iloc[i] - ema[i-1]) * multiplier) + ema[i-1]
    return ema

def MACD(DF, a=12, b=26, c=9):
    df = DF.copy()
    df["ma_fast"] = EMA(df["close"], a)
    df["ma_slow"] = EMA(df["close"], b)
    df["macd"] = df["ma_fast"] - df["ma_slow"]

    macd_series = df["macd"].dropna()
    df.loc[macd_series.index, "signal"] = EMA(macd_series, c)
    
    df["histogram"] = df["macd"] - df["signal"]
    return df[["macd", "signal", "histogram"]]


result = MACD(df)
result

,macd,signal,histogram
date,,,
2024-05-30 09:15:00+05:30,NaN,NaN,NaN
2024-05-30 09:20:00+05:30,NaN,NaN,NaN
2024-05-30 09:25:00+05:30,NaN,NaN,NaN
2024-05-30 09:30:00+05:30,NaN,NaN,NaN
2024-05-30 09:35:00+05:30,NaN,NaN,NaN
...,...,...,...
2024-06-04 15:05:00+05:30,-4.000522,-3.208465,-0.792056
2024-06-04 15:10:00+05:30,-3.800350,-3.326842,-0.473508
2024-06-04 15:15:00+05:30,-3.707904,-3.403055,-0.304849
